In [11]:
import os

In [12]:
%pwd

'C:\\Users\\asus\\Github Projects\\Chicken_Disease_Detection'

In [13]:
new_directory = r'C:\Users\asus\Github Projects\Chicken_Disease_Detection'

# Change the current working directory
os.chdir(new_directory)

# Verify the new working directory
print("Current working directory:", os.getcwd())

Current working directory: C:\Users\asus\Github Projects\Chicken_Disease_Detection


In [14]:
%pwd

'C:\\Users\\asus\\Github Projects\\Chicken_Disease_Detection'

In [15]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir : Path

In [16]:
from Chicken_Disease_Classification.constants import *
from Chicken_Disease_Classification.utils.common import read_yaml, create_directories

In [17]:
#Making the Configuration manager to create a path by calling fn from common.py
class ConfigurationManager:
    """This class is designed to read configuration and parameter files from YAML files, 
    create necessary directories, and provide access to 
    specific configuration values."""
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [18]:
import os
import urllib.request as request
import zipfile
from Chicken_Disease_Classification import logger
from Chicken_Disease_Classification.utils.common import get_size


In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2023-11-02 15:25:44,822: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-02 15:25:44,830: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-02 15:25:44,835: INFO: common: created directory at: artifacts]
[2023-11-02 15:25:44,838: INFO: common: created directory at: artifacts/data_ingestion]


[2023-11-02 15:25:49,578: INFO: 3133232766: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11605269
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "afe1cebbc8bde05cae11382d24fb25808de2f9b2388159aadb97f755936e1ae0"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9D42:353151:C3B6A:F74BB:65437203
Accept-Ranges: bytes
Date: Thu, 02 Nov 2023 09:55:47 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1269-QPG
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1698918948.894738,VS0,VE1
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 11cf8ec021e9826adc6957cee3fbb483dc7c8a66
Expires: Thu, 02 Nov 2023 10:00:47 GMT
Source-Age: 32

]
